In [1]:
from pandas import read_sql_query, read_sql_table

import pandas as pd
# import sqlite3
from matplotlib import pyplot as plt
import datetime

import requests
import json

In [8]:
import psycopg2, time
from datetime import datetime, timedelta, timezone

In [10]:
# postgres

# 1) Параметры подключения к PostgreSQL
DB = {
    "dbname": "mexc_statistics_case",
    "user": "postgres",
    "password": "1374fjsney831",
    "host": "localhost",
    "port": 5432,
}

In [4]:
# 2) Вспомогательная функция для подключения
def _pg():
    conn = psycopg2.connect(**DB)
    conn.autocommit = True
    return conn


In [5]:

# 3) Создание таблицы (если ещё нет)
def init_table() -> None:
    ddl = """
    CREATE TABLE IF NOT EXISTS pump_event_stats (
        id            SERIAL PRIMARY KEY,
        coin_symbol   TEXT NOT NULL REFERENCES coins(coin) ON DELETE CASCADE,
        pump_start_ts BIGINT NOT NULL,
        pump_end_ts   BIGINT,
        duration_sec  INTEGER,
        diff_percent  NUMERIC(8,3),
        bucket        VARCHAR(20),
        is_profit     BOOLEAN,
        UNIQUE (coin_symbol, pump_start_ts)
    );"""
    with _pg() as conn, conn.cursor() as cur:
        cur.execute(ddl)


In [6]:
# 4) Вставка/обновление одной записи пампа
def upsert_stat(symbol: str,
                ts_start: int,
                ts_end: int | None,
                diff_pct: float | None,
                bucket: str,
                is_profit: bool | None) -> None:
    duration = ts_end - ts_start if ts_end else None
    sql = """
    INSERT INTO pump_event_stats
          (coin_symbol, pump_start_ts, pump_end_ts,
           duration_sec, diff_percent, bucket, is_profit)
    VALUES (%s,%s,%s,%s,%s,%s,%s)
    ON CONFLICT (coin_symbol, pump_start_ts)
        DO UPDATE SET
            pump_end_ts   = EXCLUDED.pump_end_ts,
            duration_sec  = EXCLUDED.duration_sec,
            diff_percent  = EXCLUDED.diff_percent,
            bucket        = EXCLUDED.bucket,
            is_profit     = EXCLUDED.is_profit;"""
    with _pg() as conn, conn.cursor() as cur:
        cur.execute(sql, (symbol, ts_start, ts_end, duration,
                          diff_pct, bucket, is_profit))


In [7]:
# 5) Агрегированная статистика за последние 72 часа
def stats_last72h(symbol: str) -> dict:
    t0 = int((datetime.now(timezone.utc) - timedelta(days=3)).timestamp())
    sql = """
    WITH s AS (
        SELECT * FROM pump_event_stats
        WHERE coin_symbol=%s AND pump_start_ts >= %s
    )
    SELECT
        SUM((bucket='fifteen_minutes')::int) AS fifteen,
        SUM((bucket='three_hours')::int)     AS three_h,
        SUM((bucket='one_day')::int)         AS one_d,
        SUM((bucket='more_one_day')::int)    AS more_d,
        SUM((bucket='in_deal')::int)         AS in_deal,
        COUNT(*)                             AS total,
        SUM((is_profit IS TRUE)::int)        AS wins,
        SUM((is_profit IS FALSE)::int)       AS losses,
        ROUND(COALESCE(SUM(diff_percent)
              FILTER (WHERE is_profit),0),2) AS sum_profit_pct,
        ROUND(COALESCE(SUM(diff_percent)
              FILTER (WHERE is_profit IS FALSE),0),2) AS sum_loss_pct
    FROM s;"""
    with _pg() as conn, conn.cursor() as cur:
        cur.execute(sql, (symbol, t0))
        return dict(zip([d.name for d in cur.description], cur.fetchone()))


In [11]:
## Шаг 1 — создать таблицу `pump_event_stats`
init_table()
print('Таблица готова!')


Таблица готова!


In [12]:
## Шаг 2 — зафиксировать тестовый памп‑ивент
ts_now = int(time.time())
ts_start = ts_now - 15 * 60  # 15 минут назад
upsert_stat(
    symbol="ALPACA_USDT",
    ts_start=ts_start,
    ts_end=ts_now,
    diff_pct=8.3,
    bucket="fifteen_minutes",
    is_profit=True
)
print('✅ Запись добавлена!')


✅ Запись добавлена!


In [13]:
# Шаг 2‑bis — проверить, что запись появилась
with _pg() as conn, conn.cursor() as cur:
    cur.execute("""
        SELECT *
        FROM pump_event_stats
        WHERE coin_symbol = 'ALPACA_USDT'
        ORDER BY id DESC
        LIMIT 5
    """)
    cols = [d[0] for d in cur.description]
    for row in cur.fetchall():
        print(dict(zip(cols, row)))

{'id': 437, 'coin_symbol': 'ALPACA_USDT', 'pump_start_ts': 1746520054, 'pump_end_ts': 1746520954, 'duration_sec': 900, 'diff_percent': Decimal('8.300'), 'bucket': 'fifteen_minutes', 'is_profit': True, 'created_at': datetime.datetime(2025, 5, 6, 11, 42, 34, 498920, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}


In [ ]:
## Шаг 3 — посмотреть агрегированную статистику за 72 ч
s = stats_last72h("ALPACA_USDT")
for k, v in s.items():
    print(f"{k:15} → {v}")